<a href="https://colab.research.google.com/github/emike-akagbosu/MammoX/blob/main/Random_ForestMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from glob import glob
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from skimage.measure import label, regionprops, regionprops_table
from skimage.filters import threshold_otsu
from skimage.morphology import area_closing, area_opening
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import cv2
from google.colab.patches import cv2_imshow
import joblib
import os

In [ ]:
!python --version

Python 3.8.16


Perform Exploratory Data Analysis (EDA)

In [ ]:
data_path = "/content/drive/MyDrive/images_allCCMLO"
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#filepaths = glob.glob('{}*.png'.format(data_path), recursive=True)
filepaths = glob("/content/drive/MyDrive/images_allCCMLO/images_allCCMLO/**/*.png", recursive=True)
#filepaths = glob('{}*.png'.format(data_path))
print("Number of files: {}".format(len(filepaths)))
#fig, axis = plt.subplots(1,len(filepaths), figsize=(16,8))
#for x, ax in zip(filepaths, axis.flatten()):
#    ax.imshow(imread(x))
#    ax.set_title(x.split('\\')[1])
print("The shape of the image is:", imread(filepaths[0]).shape)

Number of files: 2780
The shape of the image is: (720, 388)


In [ ]:
#display(filepaths)
#print(filepaths[0])
#print(filepaths[500])
#print(filepaths[1000])
#print(filepaths[200])

Apply image Binarization

In [ ]:
img = imread(filepaths[200])
gray_img = rgb2gray(img)

thresh = threshold_otsu(gray_img)
display(thresh)
binarized = gray_img < thresh

binarized = binarized.astype(np.uint8)  #convert to an unsigned byte
binarized*=255

#cv2_imshow(np.array(binarized, dtype = np.uint8 ) )
#cv2.waitKey(0)

<ipython-input-25-0c9051110b6d>:2: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  gray_img = rgb2gray(img)


51

In [ ]:
#display(binarized)

In [ ]:
#[print(x) for x in binarized]

Apply morphological operations

In [ ]:
closed = area_closing(binarized)
opened = area_opening(closed)
#cv2_imshow(opened)

Label each Region using connected components

In [ ]:
label_im = label(opened)
regions = regionprops(label_im)
#cv2_imshow(label_im*255)

In [ ]:
#display(label_im)
#[print(x) for x in label_im]

Look at regions individually

In [ ]:
masks = []
bbox = []
for num, x in enumerate(regions):
    area = x.area
    convex_area = x.convex_area
    if num!=0 and x.area >= 100: 
        masks.append(regions[num].convex_image)
        bbox.append(regions[num].bbox)    
count = len(masks)
fig, axis = plt.subplots(4, int(count/4), figsize=(15,6))
for ax, box, mask in zip(axis.flatten(), bbox, masks):
    image  =  gray_img[box[0]:box[2], box[1]:box[3]] * mask
    ax.imshow(image)
    ax.axis('off')

<Figure size 1080x432 with 0 Axes>

Get the properties of each region

In [ ]:
properties = ['area','convex_area','bbox_area',
              'major_axis_length', 'minor_axis_length', 
              'perimeter', 'equivalent_diameter',
              'mean_intensity', 'solidity', 'eccentricity']
pd.DataFrame(regionprops_table(gray_img, gray_img, 
                               properties=properties)).head(10)

,area,convex_area,bbox_area,major_axis_length,minor_axis_length,perimeter,equivalent_diameter,mean_intensity,solidity,eccentricity
0,1,1,1,0.000000,0.000000,0.000000,1.128379,4.0,1.000000,0.000000
1,2,6,6,10.000000,0.000000,0.000000,1.595769,5.0,0.333333,1.000000
2,2,3,3,4.000000,0.000000,0.000000,1.595769,7.0,0.666667,1.000000
3,1,1,1,0.000000,0.000000,0.000000,1.128379,8.0,1.000000,0.000000
4,8,95,130,16.478779,11.486942,0.000000,3.191538,9.0,0.084211,0.716998
5,19,131,228,22.415520,9.323600,4.828427,4.918491,10.0,0.145038,0.909390
6,26,176,256,18.704623,13.320173,2.414214,5.753627,11.0,0.147727,0.702045
7,41,425,792,27.056669,14.434936,23.278175,7.225152,12.0,0.096471,0.845795
8,51,303,576,23.246144,13.325015,28.106602,8.058239,13.0,0.168317,0.819406
9,41,295,578,30.060496,13.777881,11.449747,7.225152,14.0,0.138983,0.888778


Combine into a single function

In [ ]:
def get_properties(name):
    filepaths = glob("/content/drive/MyDrive/images_allCCMLO/images_allCCMLO/train/{}MLO/*".format(name), recursive=True)
    print(len(filepaths))
    properties = ['area','convex_area',
                 'bbox_area','major_axis_length', 
                 'minor_axis_length', 'perimeter',  
                 'equivalent_diameter', 'mean_intensity',  
                 'solidity', 'eccentricity']
    dataframe = pd.DataFrame(columns=properties)
    for file in filepaths:
        grayscale = rgb2gray(imread(file))
        threshold = threshold_otsu(grayscale)
        binarized = grayscale < threshold         
        closed = area_closing(binarized,1000)
        opened = area_opening(closed,1000)
        labeled = label(opened)
        regions = regionprops(labeled)
        data = pd.DataFrame(regionprops_table(grayscale, grayscale,
                            properties=properties))
        data = data[(data.index!=0) & (data.area>100)]
        dataframe = pd.concat([dataframe, data])
    return dataframe

Extract dataframe for Density 1

In [ ]:
density1 = get_properties('1')
density1['type'] = '1'
print("The shape of the dataframe is: ", density1.shape)
density1.head()


194


<ipython-input-31-00e8642f1cd8>:11: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  grayscale = rgb2gray(imread(file))


The shape of the dataframe is:  (28159, 11)


,area,convex_area,bbox_area,major_axis_length,minor_axis_length,perimeter,equivalent_diameter,mean_intensity,solidity,eccentricity,type
7,104,5489,12045,206.403167,35.445667,55.006097,11.507255,8.0,0.018947,0.985144,1
8,147,6192,13157,257.175979,40.244381,71.390873,13.680870,9.0,0.023740,0.987680,1
9,174,6414,13275,281.629817,40.615427,104.390873,14.884343,10.0,0.027128,0.989546,1
10,227,7293,14720,312.690505,46.401166,146.704581,17.000746,11.0,0.031126,0.988928,1
11,203,6806,13688,317.183583,47.188387,112.982756,16.076928,12.0,0.029827,0.988871,1


In [ ]:
#display(density1)

Extract dataframe for Density 2

In [ ]:
density2 = get_properties('2')
density2['type'] = '2'
print("The shape of the dataframe is: ", density2.shape)
density2.head()

504


<ipython-input-31-00e8642f1cd8>:11: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  grayscale = rgb2gray(imread(file))


The shape of the dataframe is:  (73764, 11)


,area,convex_area,bbox_area,major_axis_length,minor_axis_length,perimeter,equivalent_diameter,mean_intensity,solidity,eccentricity,type
5,104,12298,77520,201.840868,33.631610,66.834524,11.507255,7.0,0.008457,0.986020,2
7,166,16508,75344,507.369006,53.436976,110.461941,14.538149,9.0,0.010056,0.994438,2
8,217,21574,76713,557.956177,63.289619,155.775649,16.622063,10.0,0.010058,0.993546,2
9,276,22221,75894,613.605751,67.038594,194.539105,18.746043,11.0,0.012421,0.994014,2
10,241,54376,90763,579.087942,118.345573,163.568542,17.517155,12.0,0.004432,0.978895,2


Same for 3

In [ ]:
density3 = get_properties('3')
density3['type'] = '3'
print("The shape of the dataframe is: ", density3.shape)
density3.head()

386


<ipython-input-31-00e8642f1cd8>:11: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  grayscale = rgb2gray(imread(file))


The shape of the dataframe is:  (59827, 11)


,area,convex_area,bbox_area,major_axis_length,minor_axis_length,perimeter,equivalent_diameter,mean_intensity,solidity,eccentricity,type
2,382,5639,7476,267.744942,24.131873,95.047727,22.053968,3.0,0.067743,0.995930,3
3,1091,6447,8400,282.217109,25.563480,550.422402,37.270690,4.0,0.169226,0.995889,3
4,2123,98432,187767,300.333838,84.545703,1737.420490,51.991226,5.0,0.021568,0.959560,3
5,1869,121430,194421,321.383220,229.392710,1383.856998,48.782012,6.0,0.015392,0.700383,3
6,814,139361,195960,844.954330,324.695885,257.273124,32.193431,7.0,0.005841,0.923218,3


And 4

In [ ]:
density4 = get_properties('4')
density4['type'] = '4'
print("The shape of the dataframe is: ", density4.shape)
density4.head()

298


<ipython-input-31-00e8642f1cd8>:11: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  grayscale = rgb2gray(imread(file))


The shape of the dataframe is:  (45739, 11)


,area,convex_area,bbox_area,major_axis_length,minor_axis_length,perimeter,equivalent_diameter,mean_intensity,solidity,eccentricity,type
2,174,45719,105124,1147.409914,99.362675,61.905592,14.884343,13.0,0.003806,0.996243,4
3,381,54552,113400,1089.277069,118.881278,232.373629,22.025083,14.0,0.006984,0.994027,4
4,617,62147,135632,967.687424,140.335624,396.326948,28.028357,15.0,0.009928,0.989428,4
5,731,66347,140295,902.107808,149.055322,467.232539,30.508001,16.0,0.011018,0.986255,4
6,760,66463,143514,881.688025,174.656268,400.847763,31.107267,17.0,0.011435,0.980183,4


Combine the information into a single data frame

In [ ]:
df = pd.concat([density1, density2, density3, density4])

In [ ]:
#display(df)

In [ ]:
df['ratio_length'] = (df['major_axis_length'] / 
                      df['minor_axis_length'])
df['perimeter_ratio_major'] = (df['perimeter'] /  
                               df['major_axis_length'])
df['perimeter_ratio_minor'] = (df['perimeter'] /
                               df['minor_axis_length'])
df['area_ratio_convex'] = df['area'] / df['convex_area']
df['area_ratio_bbox'] = df['area'] / df['bbox_area']
df['peri_over_dia'] = df['perimeter'] / df['equivalent_diameter']
final_df = df[df.drop('type', axis=1).columns].astype(float)
final_df = final_df.replace(np.inf, 0)
final_df['type'] = df['type']

In [ ]:
#display(final_df)

Split the training and test set

In [ ]:
X = final_df.drop('type', axis=1)
y = final_df['type']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                         test_size=0.25, 
                                         stratify = y, 
                                         random_state=1337)

In [ ]:
#display(X)

Train a random forest classifier model

In [ ]:
RF = RandomForestClassifier(max_depth=35, n_estimators=160)
RF.fit(X_train, y_train)
y_pred_RF= RF.predict(X_test)
df = pd.DataFrame()
cols = ['Machine Learning Classification Method',
        'Train Accuracy', 
        'Test Accuracy', 
        "Top 1 Feature Predictor",
        "Top 2 Feature Predictor",
        "Top 3 Feature Predictor",
        "Top 4 Feature Predictor",
        "Top 5 Feature Predictor"]
df.loc['Machine Learning Classification Method', 
       'Details'] = 'Random Forest Classifier'
df.loc['Train Accuracy', 'Details'] = RF.score(X_train, y_train) 
df.loc['Test Accuracy', 'Details'] = RF.score(X_test, y_test)
importances = (X.columns[np.argsort(RF.feature_importances_)][-5:])
df.loc['Top 1 Feature Predictor', 'Details'] = importances[4]
df.loc['Top 2 Feature Predictor', 'Details'] = importances[3]
df.loc['Top 3 Feature Predictor', 'Details'] = importances[2]
df.loc['Top 4 Feature Predictor', 'Details'] = importances[1]
df.loc['Top 5 Feature Predictor', 'Details'] = importances[0]
display(df)

,Details
Machine Learning Classification Method,Random Forest Classifier
Train Accuracy,0.999994
Test Accuracy,0.707189
Top 1 Feature Predictor,bbox_area
Top 2 Feature Predictor,mean_intensity
Top 3 Feature Predictor,convex_area
Top 4 Feature Predictor,minor_axis_length
Top 5 Feature Predictor,major_axis_length


In [ ]:
import joblib
joblib.dump(RF, "/content/drive/MyDrive/images_allCCMLO/images_allCCMLO/random_forest_MLO.joblib")

['/content/drive/MyDrive/images_allCCMLO/images_allCCMLO/random_forest_MLO.joblib']

In [ ]:
import os
joblib.dump(RF, "/content/drive/MyDrive/images_allCCMLO/images_allCCMLO/RF_compressed_MLO.joblib", compress=3)  # compression is ON!
print(f"Compressed Random Forest: {np.round(os.path.getsize('RF_compressed_MLO.joblib') / 1024 / 1024, 2) } MB")

FINAL PREDICTOR

In [ ]:
rootdir ='/content/drive/MyDrive/images_allCCMLO/images_allCCMLO/test/4/A_0494_1.LEFT_MLO.png'

In [ ]:
def get_propertiesf(rootdir):
    file = rootdir
    properties = ['area','convex_area',
                 'bbox_area','major_axis_length', 
                 'minor_axis_length', 'perimeter',  
                 'equivalent_diameter', 'mean_intensity',  
                 'solidity', 'eccentricity']
    dataframe = pd.DataFrame(columns=properties)
    grayscale = rgb2gray(imread(file))
    threshold = threshold_otsu(grayscale)
    binarized = grayscale < threshold         
    closed = area_closing(binarized,1000)
    opened = area_opening(closed,1000)
    labeled = label(opened)
    regions = regionprops(labeled)
    data = pd.DataFrame(regionprops_table(grayscale, grayscale,
                        properties=properties))
    data = data[(data.index!=0) & (data.area>100)]
    dataframe = pd.concat([dataframe, data])
    return dataframe

In [ ]:
density = get_propertiesf(rootdir)
density['type'] = 'unknown'
print("The shape of the dataframe is: ", density.shape)
display(density)



<ipython-input-84-a5b8ce6be386>:9: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  grayscale = rgb2gray(imread(file))


The shape of the dataframe is:  (172, 11)


,area,convex_area,bbox_area,major_axis_length,minor_axis_length,perimeter,equivalent_diameter,mean_intensity,solidity,eccentricity,type
5,143,2493,3024,129.470222,22.546811,21.313708,13.493452,7.0,0.057361,0.984720,unknown
6,262,2763,3168,128.068670,22.658801,96.118795,18.264412,8.0,0.094824,0.984224,unknown
7,487,3052,3288,146.932226,25.361033,216.651804,24.901158,9.0,0.159567,0.984991,unknown
8,600,2961,3105,147.224149,24.807388,349.741161,27.639532,10.0,0.202634,0.985702,unknown
9,653,3289,3600,158.976731,25.870824,389.847763,28.834449,11.0,0.198541,0.986670,unknown
...,...,...,...,...,...,...,...,...,...,...,...
173,186,42223,99528,270.931200,120.721080,30.106602,15.389040,175.0,0.004405,0.895243,unknown
174,164,43777,104430,306.231887,148.323316,28.935029,14.450304,176.0,0.003746,0.874874,unknown
175,138,40216,96945,252.575828,125.897870,27.349242,13.255454,177.0,0.003431,0.866915,unknown
176,114,13291,25404,225.374440,88.244835,8.656854,12.047793,178.0,0.008577,0.920158,unknown


In [ ]:
dff = density
dff['ratio_length'] = (dff['major_axis_length'] / 
                      dff['minor_axis_length'])
dff['perimeter_ratio_major'] = (dff['perimeter'] /  
                               dff['major_axis_length'])
dff['perimeter_ratio_minor'] = (dff['perimeter'] /
                               dff['minor_axis_length'])
dff['area_ratio_convex'] = dff['area'] / dff['convex_area']
dff['area_ratio_bbox'] = dff['area'] / dff['bbox_area']
dff['peri_over_dia'] = dff['perimeter'] / dff['equivalent_diameter']
final_dff = dff[dff.drop('type', axis=1).columns].astype(float)
final_dff = final_dff.replace(np.inf, 0)
X = final_dff
display(X)

,area,convex_area,bbox_area,major_axis_length,minor_axis_length,perimeter,equivalent_diameter,mean_intensity,solidity,eccentricity,ratio_length,perimeter_ratio_major,perimeter_ratio_minor,area_ratio_convex,area_ratio_bbox,peri_over_dia
5,143.0,2493.0,3024.0,129.470222,22.546811,21.313708,13.493452,7.0,0.057361,0.984720,5.742285,0.164622,0.945309,0.057361,0.047288,1.579559
6,262.0,2763.0,3168.0,128.068670,22.658801,96.118795,18.264412,8.0,0.094824,0.984224,5.652050,0.750525,4.242007,0.094824,0.082702,5.262627
7,487.0,3052.0,3288.0,146.932226,25.361033,216.651804,24.901158,9.0,0.159567,0.984991,5.793622,1.474502,8.542704,0.159567,0.148114,8.700471
8,600.0,2961.0,3105.0,147.224149,24.807388,349.741161,27.639532,10.0,0.202634,0.985702,5.934690,2.375569,14.098266,0.202634,0.193237,12.653657
9,653.0,3289.0,3600.0,158.976731,25.870824,389.847763,28.834449,11.0,0.198541,0.986670,6.145020,2.452232,15.069012,0.198541,0.181389,13.520209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,186.0,42223.0,99528.0,270.931200,120.721080,30.106602,15.389040,175.0,0.004405,0.895243,2.244274,0.111123,0.249390,0.004405,0.001869,1.956366
174,164.0,43777.0,104430.0,306.231887,148.323316,28.935029,14.450304,176.0,0.003746,0.874874,2.064624,0.094487,0.195081,0.003746,0.001570,2.002382
175,138.0,40216.0,96945.0,252.575828,125.897870,27.349242,13.255454,177.0,0.003431,0.866915,2.006196,0.108281,0.217234,0.003431,0.001423,2.063245
176,114.0,13291.0,25404.0,225.374440,88.244835,8.656854,12.047793,178.0,0.008577,0.920158,2.553967,0.038411,0.098100,0.008577,0.004487,0.718543


In [ ]:
loaded_rf = joblib.load("/content/drive/MyDrive/images_allCCMLO/images_allCCMLO/RF_compressed_MLO.joblib")
print(loaded_rf.predict(X))
final_output = loaded_rf.predict(X)
sum = 0
length = len(final_output)
for x in final_output:
  sum = sum +int(x)
print(sum/length)
final_predict = round(sum/length)
print(round(sum/length))

['3' '2' '2' '2' '2' '4' '4' '2' '3' '2' '2' '4' '4' '4' '4' '4' '4' '4'
 '4' '4' '4' '4' '4' '4' '4' '4' '4' '4' '4' '4' '4' '4' '4' '4' '4' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '4' '4' '4' '2' '4' '4' '3' '3' '3' '3'
 '4' '4' '2' '2' '3' '2' '2' '2' '2' '2' '2' '3' '2' '3' '3' '3' '3' '3'
 '3' '2' '3' '4' '2' '3' '3' '3' '3' '2' '3' '4' '3' '3' '3' '3' '3' '3'
 '2' '3' '3' '3' '3' '3' '3' '3' '2' '3' '2' '3' '1' '1' '2' '2' '1' '4'
 '2' '4' '2' '4' '2' '2' '2' '2' '2' '2' '2' '2' '3' '2' '2' '2' '2' '3'
 '3' '3' '3' '2' '2' '2' '2' '2' '3' '3' '2' '3' '2' '3' '2' '3' '3' '2'
 '2' '3' '3' '2' '3' '3' '1' '3' '2' '2' '2' '3' '3' '3' '3' '2' '2' '2'
 '2' '2' '2' '3' '3' '3' '2' '2' '4' '3']
2.7732558139534884
3
